In [1]:
# QUICKSTART : Working with data

In [91]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [92]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [93]:
batch_size = 64

training_loader = DataLoader(training_data, batch_size)
test_loader = DataLoader(test_data, batch_size)

In [125]:
for data,label in training_loader:
    
    print('Shape of [b,c,h,w] :',data.shape)
    print(label.shape)
    break

Shape of [b,c,h,w] : torch.Size([64, 1, 28, 28])
torch.Size([64])


In [95]:
## Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [128]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()
        self.stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.stack(x)
        return logits


model = NeuralNetwork().to(device)

model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [97]:
x = torch.rand(64, 1, 28, 28).to(device)
model(x).shape

torch.Size([64, 10])

In [98]:
# Optimizing the Model Parameters

In [99]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [121]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 500 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [122]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batch = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batch
    correct /= size

    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [124]:
epochs = 30
for epoch in range(epochs):
    print(f"epoch : {epoch + 1}\n----------------------------------")
    train(training_loader,model,loss_func,optimizer)
    test(test_loader, model, loss_func)

epoch : 1
----------------------------------
loss: 0.670434  [   64/60000]
loss: 0.656236  [32064/60000]
Test Error: 
 Accuracy: 75.3%, Avg loss: 0.688533 

epoch : 2
----------------------------------
loss: 0.648341  [   64/60000]
loss: 0.643358  [32064/60000]
Test Error: 
 Accuracy: 76.1%, Avg loss: 0.674012 

epoch : 3
----------------------------------
loss: 0.628168  [   64/60000]
loss: 0.632013  [32064/60000]
Test Error: 
 Accuracy: 76.7%, Avg loss: 0.660535 

epoch : 4
----------------------------------
loss: 0.609737  [   64/60000]
loss: 0.621943  [32064/60000]
Test Error: 
 Accuracy: 77.3%, Avg loss: 0.648019 

epoch : 5
----------------------------------
loss: 0.592824  [   64/60000]
loss: 0.612933  [32064/60000]
Test Error: 
 Accuracy: 77.8%, Avg loss: 0.636411 

epoch : 6
----------------------------------
loss: 0.577337  [   64/60000]
loss: 0.604809  [32064/60000]
Test Error: 
 Accuracy: 78.1%, Avg loss: 0.625655 

epoch : 7
----------------------------------
loss: 0.56306

In [126]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")


Saved PyTorch Model State to model.pth


In [129]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [130]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
